In [2]:
import math
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import evaluate
from datasets import DatasetDict, Dataset
import numpy as np
import random
import os
# import wandb

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
# wandb.init(project="testing-ts-training")


In [4]:
ds = None
train_split = 0.8
validation_split = 0.2
with open("final_lyrics.txt", "r") as f:
    arr = f.readlines()
    train, validation, test = np.split(arr, [int(len(arr)*train_split), int(len(arr)*validation_split)])
    datasets = DatasetDict(
      {
          'train': Dataset.from_dict({'text': list(train)}),
          'validation': Dataset.from_dict({'text': list(validation)}),
          'test': Dataset.from_dict({'text': list(test)})
      }
      
  )
    


In [5]:
def tokenizer_func(data):
    return tokenizer(data["text"], truncation=True)
    
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

block_size = 128

tokenized_datasets = datasets.map(tokenizer_func, batched=True, remove_columns=["text"], num_proc=4)
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=4,
    batch_size=1000,
)

# tokenizer.decode(lm_datasets["train"][1]["input_ids"])

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
# training_args = TrainingArguments(output_dir="output/ts-testing-training-model",evaluation_strategy="epoch",learning_rate=2e-5,weight_decay=0.01, num_train_epochs=25, report_to='wandb', use_mps_device=True, eval_steps=1, logging_steps=4)
# trainer = Trainer(model=model, args=training_args, train_dataset=lm_datasets['train'], eval_dataset=lm_datasets['test'], data_collator=data_collator)

tokenizer.save_pretrained("output/ts-song-generation")

# trainer.train()

tokenizer config file saved in output/ts-song-generation/tokenizer_config.json
Special tokens file saved in output/ts-song-generation/special_tokens_map.json


('output/ts-song-generation/tokenizer_config.json',
 'output/ts-song-generation/special_tokens_map.json',
 'output/ts-song-generation/vocab.json',
 'output/ts-song-generation/merges.txt',
 'output/ts-song-generation/added_tokens.json',
 'output/ts-song-generation/tokenizer.json')

wandb: Network error (ReadTimeout), entering retry loop.


In [9]:
eval_results = trainer.evaluate()

print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 295
  Batch size = 8
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/37 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: bhavyamuni. Use `wandb login --relogin` to force relogin


Perplexity: 26.96
